# AKS Cookbook

## 🧪 AKS Quickstart with Azure CLI lab

Use this lab to quickly provision an AKS cluster with the Azure CLI and deploy a sample application to explore Kubernetes concepts. It's based on the official [AKS documentation](https://learn.microsoft.com/en-us/azure/aks/learn/quick-kubernetes-deploy-cli).  

▶️ Click on the `Run All` button to execute all the subsequent steps in sequence, or run each step individually by executing the cells one at a time.

### TOC

- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Verify the Azure CLI and connected Azure subscription](#1)
- [2️⃣ Create a new Azure Resource Group or reuse an existing one](#2)
- [3️⃣ Create an AKS cluster](#3)
- [4️⃣ Connect to the AKS cluster](#4)
- [5️⃣ Retrieve the list of AKS cluster nodes](#5)
- [6️⃣ Deploy the sample application](#6)
- [7️⃣ List the Kubernetes resources created](#7)
- [8️⃣ Test the application](#8)
- [9️⃣ Observe the logs](#9)
- [🗑️ Clean up resources](#clean)


<a id='0'></a>
### 0️⃣ Initialize notebook variables
Adjust the location parameters according your preferences and on the [product availability by Azure region](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable).

In [ ]:
import os, subprocess, datetime, time, json, requests

create_resources = True # specify True if you want to create new resources, False to use existing ones

if create_resources:
    # create new resources with the following properties
    deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
    resource_group_name = f"lab-{deployment_name}" # change the name to match your naming convention
    resource_group_location = "eastus2"
    aks_resource_name = "aks-cluster"
    aks_node_count = 1
else:
    # or use the following existing resources
    resource_group_name = ""
    aks_resource_name = ""



In [ ]:
# helper functions
print_ok = lambda message, output='', duration='': print("✅ \x1b[1;32m", message, "\x1b[0m⌚", datetime.datetime.now().time(), duration, "\n", output)
print_info = lambda message, output='', duration='': print("👉🏽 \x1b[1;34m", message, "\x1b[0m⌚", datetime.datetime.now().time(), duration, "\n", output)
print_error = lambda message, output='', duration='': print("⛔ \x1b[1;31m", message, "\x1b[0m⌚", datetime.datetime.now().time(), duration, "\n", output)
print_warning = lambda message, output='', duration='': print("⚠️ \x1b[1;33m", message, "\x1b[0m⌚", datetime.datetime.now().time(), duration, "\n", output)
class Output(object):
    def __init__(self, success, text):
        self.success = success
        self.text = text
        try:
            self.json_data = json.loads(text)
        except:
            self.json_data = None
def run(command, ok_message = '', error_message = '', print_output = False):
    start_time = time.time()
    try:
        output_text = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT).decode("utf-8")
        success = True
    except subprocess.CalledProcessError as e:
        output_text = e.output.decode("utf-8")
        success = False
    minutes, seconds = divmod(time.time() - start_time, 60)
    print_message = print_ok if success else print_error
    if (ok_message or error_message):
        print_message(ok_message if success else error_message, output_text if not success or print_output  else "", f"[{int(minutes)}m:{int(seconds)}s]")
    return Output(success, output_text)

print_info('Notebook initiaized')

<a id='1'></a>
### 1️⃣ Verify the Azure CLI and connected Azure subscription
The following commands ensure that you have the latest version of the Azure CLI and relevant extensions installed while also verifying that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = run("az upgrade", "Azure CLI is up to date", "Failed to upgrade Azure CLI")
output = run("az account show", "Retrieved az account", "Failed to get the current az account")
if output.success and output.json_data:
    current_user = output.json_data.get('user').get('name')
    subsciption_id = output.json_data.get('id')
    tenant_id = output.json_data.get('tenantId')
output = run("az provider register --namespace Microsoft.ContainerService --wait", "Microsoft.ContainerService registered in your subscription", "Failed to register Microsoft.ContainerService")
output = run("az provider register --namespace Microsoft.KubernetesConfiguration --wait", "Microsoft.KubernetesConfiguration registered in your subscription", "Failed to register Microsoft.KubernetesConfiguration")
output = run("az extension add --name k8s-extension", "az k8s-extension installed", "Failed to install az k8s-extension")
output = run("az extension update --name k8s-extension", "az k8s-extension updated", "Failed to update az k8s-extension")
output = run("az extension add --name aks-preview", "az aks-preview extension installed", "Failed to install az aks-preview extension")
output = run("az extension update --name aks-preview", "az aks-preview extension updated", "Failed to update az aks-preview extension")


<a id='2'></a>
### 2️⃣ Create a new Azure Resource Group or reuse an existing one
All resources deployed in this lab will be created within the designated resource group. 

In [ ]:
if not resource_group_name:
    print_warning('Please specify the resource group name')
else:
    output = run(f"az group show --name {resource_group_name}")
    if create_resources:    
        if output.success:
            print_info(f"Using existing resource group '{resource_group_name}'")
        else:
            output = run(f"az group create --name {resource_group_name} --location {resource_group_location}", 
                                    f"Resource group '{resource_group_name}' created", 
                                    f"Failed to create the resource group '{resource_group_name}'")
    else:
        if output.success:
            print_info(f"Using resource group '{resource_group_name}'")
        else:
            print_error(f"Resource group '{resource_group_name}' does not exist")


<a id='3'></a>
### 3️⃣ Create an AKS cluster
The following step creates an AKS cluster using the [az aks create](https://learn.microsoft.com/en-us/cli/azure/aks?view=azure-cli-latest#az-aks-create) command. 

In [ ]:
output = run(f"az aks create --resource-group {resource_group_name} --name {aks_resource_name} --node-count {aks_node_count} --generate-ssh-keys",
             f"AKS cluster '{aks_resource_name}' created",
             f"Failed to create AKS cluster '{aks_resource_name}'")

<a id='4'></a>
### 4️⃣ Connect to the AKS cluster
Configure kubectl to connect to your Kubernetes cluster using the [az aks get-credentials](https://learn.microsoft.com/en-us/cli/azure/aks?view=azure-cli-latest#az-aks-get-credentials) command. This command downloads credentials and configures the Kubernetes CLI to use them.

In [ ]:
output = run("az aks install-cli", "kubectl installed", "Failed to install kubectl")
output = run(f"az aks get-credentials --resource-group {resource_group_name} --name {aks_resource_name} --overwrite-existing",
             f"Credentials for AKS cluster '{aks_resource_name}' configured",
             f"Failed to configure credentials for AKS cluster '{aks_resource_name}'")


<a id='5'></a>
### 5️⃣ Retrieve the list of AKS cluster nodes
Verify the connection to your cluster using the [kubectl get](https://kubernetes.io/docs/reference/generated/kubectl/kubectl-commands#get) command. This command returns a list of the cluster nodes.

In [ ]:
! kubectl get nodes

<a id='6'></a>
### 6️⃣ Deploy the sample application
To deploy the application, you use a manifest file to create all the objects required to run the [AKS Store application](https://github.com/Azure-Samples/aks-store-demo). A [Kubernetes manifest file](https://learn.microsoft.com/en-us/azure/aks/core-aks-concepts#deployments-and-yaml-manifests) defines a cluster's desired state, such as which container images to run.  
  
The manifest includes the following Kubernetes deployments and services:
- Store front: Web application for customers to view products and place orders.
- Product service: Shows product information.
- Order service: Places orders.
- Rabbit MQ: Message queue for an order queue.

Review the [sample manifest file](aks-store-quickstart.yaml) that will be used in the following command. 

In [ ]:
! kubectl apply -f aks-store-quickstart.yaml


<a id='7'></a>
### 7️⃣ List the Kubernetes resources created
The following commands list the deployed pods and services in the AKS cluster.

In [ ]:
output = run("kubectl get pods -n default", "Pods in the default namespace", "Failed to get pods in the default namespace")
print(output.text) 
output = run("kubectl get service -n default", "Services in the default namespace", "Failed to get services in the default namespace")
print(output.text) 


<a id='8'></a>
### 8️⃣ Test the application
The following script verifies the readiness of the `store-front` pod and retrieves the external IP address for the `store-front` service, enabling you to test the application UI.

In [ ]:
from IPython.display import HTML
wait_time = 60 # in seconds
sleep_time = 5 # in seconds
request_timeout = 60 # in seconds
message = "<p>Application is not ready yet. Check the logs.</p>"
while wait_time - sleep_time > 0:
    output = run("kubectl get pods -l app=store-front -o jsonpath={..status.phase}")
    if output.success and output.text == "Running":
        output = run("kubectl get service store-front --output jsonpath={..status.loadBalancer.ingress[0]}")
        if output.success and output.json_data:
            ingress_ip = output.json_data.get('ip')
            print_info(f"Service External IP: {ingress_ip}")
            try:
                response = requests.get(f"http://{ingress_ip}", timeout=request_timeout)
                if (response.status_code == 200):
                    message = f"<p>Test the application by opening the following link:</p><a href='http://{ingress_ip}' target='_blank'>http://{ingress_ip}</a>"
                    break
            except:
                message = "<p>The application is currently unresponsive...</p>"
    wait_time -= sleep_time
    time.sleep(sleep_time)
HTML(message)

<a id='9'></a>
### 9️⃣ Observe the logs
When operating an Azure Kubernetes Service (AKS) cluster, you may need to review logs to troubleshoot a problem. Azure portal has a built-in capability that allows you to view logs for AKS main components and cluster containers.  
The following commands display logs for each pod that is part of the sample application.

In [ ]:
output = run("kubectl logs -l app=rabbitmq --tail=3", "Logs for the rabbitmq pod", "Error in logs for the rabbitmq pod")
print(output.text) 
output = run("kubectl logs -l app=order-service --tail=3", "Logs for the order-service pod", "Error in logs for the order-service pod")
print(output.text) 
output = run("kubectl logs -l app=product-service --tail=3", "Logs for the product-service pod", "Error in logs for the product-service pod")
print(output.text) 
output = run("kubectl logs -l app=store-front --tail=3", "Logs for the store-front pod", "Error in logs for the store-front pod")
print(output.text) 


<a id='clean'></a>
### 🗑️ Clean up resources
When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered. Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.